<a href="https://colab.research.google.com/github/JulienVm/NLP_Project/blob/main/NLP_Project_Julien_VAN_MIEGEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projet NLP - Analyse de sentiments



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
from nltk.tokenize import word_tokenize
from nltk.text import Text
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
import nltk

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

# Création du set de données

In [101]:
nRowsRead = 1000 # le nombre de lignes qu'on lit dans notre csv
data=pd.read_csv("drive/MyDrive/NLP_data.csv", encoding="ISO-8859-1", nrows=nRowsRead, header=None, names=['target','id','date','flag','user','text'], dtype={'target':np.int32,'id':np.int32,'date':np.string_,'flag':np.string_,'user':np.string_,"text":np.string_})
data.head() # on a un dataframe à 6 colonnes : le sentiment entre 0 et 4, l'id, la date, le flag, l'utilisateur et le texte

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


# Mise en forme des données

In [102]:
feelingList=data['target']
tweetList=data['text']
tweetList[0:5]

0    @switchfoot http://twitpic.com/2y1zl - Awww, t...
1    is upset that he can't update his Facebook by ...
2    @Kenichan I dived many times for the ball. Man...
3      my whole body feels itchy and like its on fire 
4    @nationwideclass no, it's not behaving at all....
Name: text, dtype: object

Formation de vecteurs de mots

In [103]:
from nltk.stem.porter import *
stemmer=stemmer = PorterStemmer()
from nltk.stem import WordNetLemmatizer
Word_Lemmatizer = WordNetLemmatizer()

for i in range(len(tweetList)):
  tweet=""
  for char in tweetList[i]:
    if char not in [',','!','(',')','?',':','[',']',';','-','+']: # on remplace la ponctuation par des points
      tweet+=char
    else:
      tweet+='.'
  tweet=tweet.lower() # on met en minuscule
  tweet=word_tokenize(tweet) # tokenisation
  tweet=[w for w in tweet if not w in list(nltk.corpus.stopwords.words('english'))] # on enlève les stopwords
  tweet=[stemmer.stem(w) for w in tweet] #stemming
  tweet=[Word_Lemmatizer.lemmatize(w) for w in tweet] # lemmatisation
  tweet=[w for w in tweet if ('@' not in w) and ('http' not in w)] # on enlève certains mots jugés inutiles (mentions, liens...)
  tweetList[i]=tweet
tweetList[0:5]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0    [switchfoot, ., awww, ., 's, bummer, ., should...
1    [upset, ca, n't, updat, facebook, text, ..., m...
2    [kenichan, dive, mani, time, ball, ., manag, s...
3               [whole, bodi, feel, itchi, like, fire]
4    [nationwideclass, ., 's, behav, ., 'm, mad, .,...
Name: text, dtype: object

Création du set de vocabulaire

In [104]:
V=set()
for tweet in tweetList :
  for word in tweet :
    V.add(word)
len(V)

3019

Création de bag of words

In [105]:
list_bags=[]
for tweet in tweetList:
  bag=np.zeros(len(V))
  i=0
  for word in V:
    if word in tweet :
      bag[i]=tweet.count(word)
    i+=1
  list_bags.append(bag)

list_bags=np.array(list_bags)
finalData=pd.DataFrame(list_bags,columns=V)
finalData.head()

,jemcam,lt./3,bank,wat,'look,random,goto,owl,terribl,prepatori,flower,knight,latest,furiou,favorit,today,way,alot,crummyasshol,cri,relat,diari,diff,life,djsoulsist,'ll,6.10.,no..,fell,quot.twitter,daddi,prayer,suffer,fan,cheechbud,cooooold,outag,rabbit,adamgoldston,miamiiboii,...,mani,ie,missin,time,prais,becom,least,nw,fruit,mind,video,wnt,midv,featherinair,widyatarina,total,unholyknight,joshi,quiet,exactli,icecream,8900,server,teardrop,nevadawolf,came,scare,soak,beachbassbon,ca,sighhhh,calori,help,singlet,kutner,chillen,log,professor,13th,thang
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Création du CNN

In [108]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
import seaborn as sn
from sklearn import linear_model

from sklearn.linear_model import LogisticRegression

x = finalData
y = feelingList

x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.79,random_state=0)
